In [16]:
import pandas as pd
import re
from datetime import datetime, timedelta
from networkx import betweenness_centrality, closeness_centrality, eigenvector_centrality
from collections import defaultdict


In [17]:
import pandas as pd
import re
file_path = 'transcriptions/kakao/kakaotalk_conversation.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()


def extract_data(data):
    pattern = re.compile(
        r'(\d{4}\. \d{1,2}\. \d{1,2})\. (오전|오후) \d{1,2}:\d{2}, (Speaker SPEAKER_\d{2}) : (.+)')
    matches = pattern.findall(data)

    extracted_data = []
    for match in matches:
        date_str = match[0]
        datetime_obj = datetime.strptime(date_str, '%Y. %m. %d')
        date = datetime_obj.strftime('%Y-%m-%d')
        speaker = match[2].replace('Speaker ', '')
        text = match[3]
        word_count = len(text.split())
        extracted_data.append([date, speaker, text, word_count])

    return extracted_data


extracted_data = extract_data(data)
df = pd.DataFrame(extracted_data, columns=[
                  'Date', 'Speaker', 'Text', 'word_count'])

df['Date'] = pd.to_datetime(df['Date'])


def assign_meeting_number(date):
    if '2024-05-21' <= date <= '2024-05-24':
        return 1
    elif '2024-05-25' <= date <= '2024-05-27':
        return 2
    elif date == '2024-05-28':
        return 3
    elif date == '2024-05-29':
        return 4
    elif date == '2024-05-30':
        return 5
    elif date == '2024-05-31':
        return 6
    elif '2024-06-01' <= date <= '2024-06-10':
        return 7
    elif '2024-06-11' <= date <= '2024-06-12':
        return 8
    elif '2024-06-13' <= date <= '2024-06-14':
        return 9
    elif '2024-06-15' <= date <= '2024-06-17':
        return 10
    elif '2024-06-18' <= date <= '2024-06-19':
        return 11
    else:
        return None


df['meeting_number'] = df['Date'].apply(
    lambda x: assign_meeting_number(x.strftime('%Y-%m-%d')))

print(df)

           Date             Speaker                            Text  \
0    2024-05-21  Speaker SPEAKER_03                       여러분 반가워요!   
1    2024-05-21  Speaker SPEAKER_03             준영님 노션에 번호 올려주세요 ㅋㅋ   
2    2024-05-21  Speaker SPEAKER_01                        안녕하세요 !    
3    2024-05-21  Speaker SPEAKER_00                          반갑습니다~   
4    2024-05-21  Speaker SPEAKER_04                    아 넵! 올렸습니다!    
...         ...                 ...                             ...   
2062 2024-06-19  Speaker SPEAKER_04                            에...   
2063 2024-06-19  Speaker SPEAKER_04                            흠...   
2064 2024-06-19  Speaker SPEAKER_03       이 단톡방을 텍스트로 받았을 때 얘기하는건가?   
2065 2024-06-19  Speaker SPEAKER_04  아니 내가 형들 다 저이름으로 바꾸고 추출하려고 했거덩   
2066 2024-06-19  Speaker SPEAKER_01                             뮤ㅠ야   

      word_count  meeting_number  
0              2               1  
1              5               1  
2              2               1  
3      

In [18]:


def create_dataset(df, project_number):
    dataset = []
    speaker_word_counts = df.groupby(['meeting_number', 'Speaker'])[
        'word_count'].sum().to_dict()
    total_words = df.groupby('meeting_number')['word_count'].sum().to_dict()
    meeting_dates = df.groupby('meeting_number')['Date'].agg([
        'min', 'max']).to_dict('index')

    for (meeting_number, speaker), word_count in speaker_word_counts.items():
        start_date = meeting_dates[meeting_number]['min']
        end_date = meeting_dates[meeting_number]['max']
        duration = ((end_date - start_date).days + 1) * 24

        dataset.append({
            'id': f'{project_number}_{meeting_number}_{speaker}',
            'project': project_number,
            'meeting_number': meeting_number,
            'speaker_number': int(speaker.split('_')[1]),
            'speech_frequency': word_count,
            'total_words': total_words[meeting_number],
            'duration': duration
        })
    return pd.DataFrame(dataset)


dataset_project4 = create_dataset(df, 4)

dataset_project4 = dataset_project4.sort_values(
    by=['meeting_number', 'speaker_number'])


dataset_project4

,id,project,meeting_number,speaker_number,speech_frequency,total_words,duration
0,4_1_Speaker SPEAKER_00,4,1,0,1,22,24
1,4_1_Speaker SPEAKER_01,4,1,1,2,22,24
2,4_1_Speaker SPEAKER_02,4,1,2,2,22,24
3,4_1_Speaker SPEAKER_03,4,1,3,7,22,24
4,4_1_Speaker SPEAKER_04,4,1,4,10,22,24
5,4_2_Speaker SPEAKER_00,4,2,0,132,416,72
6,4_2_Speaker SPEAKER_01,4,2,1,67,416,72
7,4_2_Speaker SPEAKER_02,4,2,2,14,416,72
8,4_2_Speaker SPEAKER_03,4,2,3,136,416,72
9,4_2_Speaker SPEAKER_04,4,2,4,67,416,72


In [19]:
dataset_project4['normalized_speech_frequency'] = dataset_project4['speech_frequency'] / \
    dataset_project4['duration']

In [20]:


def compute_interaction_frequency(df, project_number):
    interaction_records = []
    for meeting_number in df['meeting_number'].unique():
        meeting_df = df[df['meeting_number'] == meeting_number]
        interaction_counts = defaultdict(lambda: defaultdict(int))
        for i in range(len(meeting_df)):
            prev_speaker = meeting_df.iloc[i]['Speaker']
            if i < len(meeting_df) - 1:
                next_speaker = meeting_df.iloc[i + 1]['Speaker']
            else:
                next_speaker = meeting_df.iloc[i]['Speaker']
            interaction_counts[prev_speaker][next_speaker] += 1
        for prev_speaker, next_speakers in interaction_counts.items():
            for next_speaker, count in next_speakers.items():
                interaction_records.append({
                    'project': project_number,
                    'meeting_number': meeting_number,
                    'speaker_id': int(prev_speaker.split('_')[1]),
                    'next_speaker_id': int(next_speaker.split('_')[1]),
                    'count': count
                })
    return pd.DataFrame(interaction_records)


interaction_frequency_df = compute_interaction_frequency(df, 4)
interaction_frequency_df.sort_values(
    by=['meeting_number', 'speaker_id', 'next_speaker_id'])

,project,meeting_number,speaker_id,next_speaker_id,count
3,4,1,0,4,1
2,4,1,1,0,1
6,4,1,2,2,1
1,4,1,3,1,1
0,4,1,3,3,1
...,...,...,...,...,...
160,4,11,4,0,27
159,4,11,4,1,18
161,4,11,4,2,26
158,4,11,4,3,115


In [21]:
def generate_all_pairs(interaction_records, dataset):
    all_pairs = []
    for (project, meeting), group in dataset.groupby(['project', 'meeting_number']):
        speakers = group['speaker_number'].unique()
        for speaker1 in speakers:
            for speaker2 in speakers:
                if not interaction_records[(interaction_records['project'] == project) & (interaction_records['meeting_number'] == meeting) & (interaction_records['speaker_id'] == speaker1) & (interaction_records['next_speaker_id'] == speaker2)].empty:
                    continue
                all_pairs.append({
                    'project': project,
                    'meeting_number': meeting,
                    'speaker_id': speaker1,
                    'next_speaker_id': speaker2,
                    'count': 0
                })
    return pd.DataFrame(all_pairs)


all_pairs = generate_all_pairs(interaction_frequency_df, dataset_project4)
interaction_records = pd.concat(
    [interaction_frequency_df, all_pairs], ignore_index=True)
interaction_records = interaction_records.sort_values(
    by=['project', 'meeting_number', 'speaker_id', 'next_speaker_id']).reset_index(drop=True)

In [22]:
combined_dataset = pd.merge(dataset_project4, interaction_records, how='left', left_on=[
                            'project', 'meeting_number', 'speaker_number'], right_on=['project', 'meeting_number', 'speaker_id'])
combined_dataset['count'] = combined_dataset['count'].fillna(0).astype(int)

In [23]:
combined_dataset

,id,project,meeting_number,speaker_number,speech_frequency,total_words,duration,normalized_speech_frequency,speaker_id,next_speaker_id,count
0,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,0,0
1,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,1,0
2,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,2,0
3,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,3,0
4,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...
204,4_11_Speaker SPEAKER_04,4,11,4,960,5474,48,20.000000,4,0,27
205,4_11_Speaker SPEAKER_04,4,11,4,960,5474,48,20.000000,4,1,18
206,4_11_Speaker SPEAKER_04,4,11,4,960,5474,48,20.000000,4,2,26
207,4_11_Speaker SPEAKER_04,4,11,4,960,5474,48,20.000000,4,3,115


## Compute Network Density


In [24]:
import networkx as nx

meeting = [1,  2,  3, 4, 5, 6,  7,  8,  9, 10, 11]


def compute_density(G):
    num_nodes = len(G)
    if num_nodes < 2:
        return 0
    possible_edges = num_nodes * (num_nodes - 1)  # For directed graph
    actual_edges = sum(1 for u, v, data in G.edges(
        data=True) if u != v and data['weight'] > 0)
    return actual_edges / possible_edges


def weighted_density(G):
    if len(G) == 0:
        return 0
    total_weight = sum(data['weight'] for u, v, data in G.edges(data=True))
    # total_weight = sum(data['weight'] for u, v, data in G.edges(data=True) if u != v)
    num_nodes = len(G)
    max_weight = max(data['weight'] for u, v, data in G.edges(data=True))
    # max_weight = max(data['weight'] for u, v, data in G.edges(data=True) if u != v)
    possible_edges = num_nodes * (num_nodes - 1)  # For directed graph
    return total_weight / (possible_edges * max_weight) if possible_edges > 0 else 0


def create_interaction_graphs(df):
    graphs = []
    for meeting_number in df['meeting_number'].unique():
        G = nx.DiGraph()
        meeting_data = df[df['meeting_number'] == meeting_number]
        for i in range(len(meeting_data)):
            prev_speaker = f"SPEAKER_{meeting_data.iloc[i]['speaker_id']:02d}"
            next_speaker = f"SPEAKER_{meeting_data.iloc[i]['next_speaker_id']:02d}"
            count = meeting_data.iloc[i]['count']
            if count > 0:
                if G.has_edge(prev_speaker, next_speaker):
                    G[prev_speaker][next_speaker]['weight'] += count
                else:
                    G.add_edge(prev_speaker, next_speaker, weight=count)

                if G.has_edge(next_speaker, prev_speaker) and prev_speaker != next_speaker:
                    G[next_speaker][prev_speaker]['weight'] += count
                else:
                    G.add_edge(next_speaker, prev_speaker, weight=count)

        graphs.append(G)
    return graphs


interaction_graphs = create_interaction_graphs(interaction_frequency_df)
densities = [compute_density(G) for G in interaction_graphs]
weighted_density = [weighted_density(G) for G in interaction_graphs]
densities
interaction_graphs

## Compute Centralities


In [25]:
# Define centrality measures function


def compute_centralities(G):
    if len(G) == 0:
        centralities = {
            'degree_centrality': {},
            'indegree_centrality': {},
            'outdegree_centrality': {},
            'betweenness_centrality': {},
            'closeness_centrality': {},
            'eigenvector_centrality': {},
            'pagerank': {}
        }
    else:
        try:
            eigenvector_cent = eigenvector_centrality(G, max_iter=2000, weight='weight')
        except nx.PowerIterationFailedConvergence:
            eigenvector_cent = {node: 0 for node in G.nodes()}
        num_nodes = len(G)
        possible_edges = num_nodes * (num_nodes - 1)
        max_weight = max(data['weight'] for u, v, data in G.edges(data=True) if u != v)
        centralities = {
            'degree_centrality': {k: v / (possible_edges * max_weight) for k, v in dict(G.degree(weight='weight')).items()},
            'indegree_centrality': {k: v / (possible_edges * max_weight) for k, v in dict(G.in_degree(weight='weight')).items()},
            'outdegree_centrality': {k: v / (possible_edges * max_weight) for k, v in dict(G.out_degree(weight='weight')).items()},
            'betweenness_centrality': {k: v / (possible_edges * max_weight) for k, v in betweenness_centrality(G, weight='weight').items()},
            'closeness_centrality': {k: v / (possible_edges * max_weight) for k, v in closeness_centrality(G, distance='weight').items()},
            'eigenvector_centrality': {k: v / (possible_edges * max_weight) for k, v in eigenvector_cent.items()},
            'pagerank': {k: v / (possible_edges * max_weight) for k, v in nx.pagerank(G, weight='weight').items()}
        }
    return centralities


centralities = [compute_centralities(G) for G in interaction_graphs]
centralities

[{'degree_centrality': {'SPEAKER_03': 4,
   'SPEAKER_01': 4,
   'SPEAKER_00': 4,
   'SPEAKER_04': 6,
   'SPEAKER_02': 4},
  'indegree_centrality': {'SPEAKER_03': 2,
   'SPEAKER_01': 2,
   'SPEAKER_00': 2,
   'SPEAKER_04': 3,
   'SPEAKER_02': 2},
  'outdegree_centrality': {'SPEAKER_03': 2,
   'SPEAKER_01': 2,
   'SPEAKER_00': 2,
   'SPEAKER_04': 3,
   'SPEAKER_02': 2},
  'betweenness_centrality': {'SPEAKER_03': 0.0,
   'SPEAKER_01': 0.5,
   'SPEAKER_00': 0.6666666666666666,
   'SPEAKER_04': 0.5,
   'SPEAKER_02': 0.0},
  'closeness_centrality': {'SPEAKER_03': 0.4,
   'SPEAKER_01': 0.5714285714285714,
   'SPEAKER_00': 0.6666666666666666,
   'SPEAKER_04': 0.5714285714285714,
   'SPEAKER_02': 0.4},
  'eigenvector_centrality': {'SPEAKER_03': 0.1902272488362771,
   'SPEAKER_01': 0.2540175296602318,
   'SPEAKER_00': 0.4029919181287057,
   'SPEAKER_04': 0.6871123429716617,
   'SPEAKER_02': 0.5145510960870985},
  'pagerank': {'SPEAKER_03': 0.19231639367930523,
   'SPEAKER_01': 0.1896027446605047

## Add Centralities and Network Density to Combined Dataset


In [26]:
for centrality_measure in ['degree_centrality', 'indegree_centrality', 'outdegree_centrality', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality', 'pagerank']:
    combined_dataset[centrality_measure] = 0
combined_dataset['network_density'] = 0
combined_dataset['weighted_network_density'] = 0


meeting_numbers = [1, 2, 3, 6, 7, 8, 9, 10, 11]

for i, centralities in enumerate(centralities):
    density = densities[i]
    weighted_density_value = weighted_density[i]
    meeting_number = meeting_numbers[i]
    for centrality_measure, centrality_values in centralities.items():
        for node, value in centrality_values.items():
            combined_dataset.loc[
                (combined_dataset['project'] == 4) &
                (combined_dataset['meeting_number'] == meeting_number) &
                (combined_dataset['speaker_number']
                 == int(node.split('_')[1])),
                centrality_measure] = value
    combined_dataset.loc[
        (combined_dataset['project'] == 4) &
        (combined_dataset['meeting_number'] == meeting_number),
        'network_density'] = density
    combined_dataset.loc[
        (combined_dataset['project'] == 4) &
        (combined_dataset['meeting_number'] == meeting_number),
        'weighted_network_density'] = weighted_density_value


combined_dataset

C:\Users\sjy99\AppData\Local\Temp\ipykernel_28260\690569308.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_dataset.loc[
C:\Users\sjy99\AppData\Local\Temp\ipykernel_28260\690569308.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_dataset.loc[
C:\Users\sjy99\AppData\Local\Temp\ipykernel_28260\690569308.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1902272488362771' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_dataset.loc[
C:\Users\sjy99\AppData\Local\Temp\ipykernel_28260

,id,project,meeting_number,speaker_number,speech_frequency,total_words,duration,normalized_speech_frequency,speaker_id,next_speaker_id,count,degree_centrality,indegree_centrality,outdegree_centrality,betweenness_centrality,closeness_centrality,eigenvector_centrality,pagerank,network_density,weighted_network_density
0,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,0,0,4,2,2,0.666667,0.666667,0.402992,0.183218,0.4,0.4
1,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,1,0,4,2,2,0.666667,0.666667,0.402992,0.183218,0.4,0.4
2,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,2,0,4,2,2,0.666667,0.666667,0.402992,0.183218,0.4,0.4
3,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,3,0,4,2,2,0.666667,0.666667,0.402992,0.183218,0.4,0.4
4,4_1_Speaker SPEAKER_00,4,1,0,1,22,24,0.041667,0,4,1,4,2,2,0.666667,0.666667,0.402992,0.183218,0.4,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,4_11_Speaker SPEAKER_04,4,11,4,960,5474,48,20.000000,4,0,27,910,455,455,0.000000,0.017621,0.600885,0.275658,1.0,60.0
205,4_11_Speaker SPEAKER_04,4,11,4,960,5474,48,20.000000,4,1,18,910,455,455,0.000000,0.017621,0.600885,0.275658,1.0,60.0
206,4_11_Speaker SPEAKER_04,4,11,4,960,5474,48,20.000000,4,2,26,910,455,455,0.000000,0.017621,0.600885,0.275658,1.0,60.0
207,4_11_Speaker SPEAKER_04,4,11,4,960,5474,48,20.000000,4,3,115,910,455,455,0.000000,0.017621,0.600885,0.275658,1.0,60.0


## Compute Gini Coefficient


In [27]:

import numpy as np


def gini_coefficient(x):
    x = np.array(x, dtype=np.float64)
    if np.amin(x) < 0:
        x -= np.amin(x)  # values cannot be negative
    x += 0.0000001  # values cannot be 0
    x = np.sort(x)  # values must be sorted
    index = np.arange(1, x.shape[0] + 1)  # index per array element
    n = x.shape[0]
    return ((np.sum((2 * index - n - 1) * x)) / (n * np.sum(x)))


def compute_gini(df):
    gini_values = []
    meetings = df['meeting_number'].unique()
    for meeting_number in meetings:
        meeting_data = df[df['meeting_number'] == meeting_number]
        interaction_counts = [meeting_data[(meeting_data['speaker_id'] == speaker) & (
            meeting_data['speaker_id'] != meeting_data['next_speaker_id'])]['count'].sum() for speaker in meeting_data['speaker_id'].unique()]
        gini_values.append(gini_coefficient(interaction_counts))
    return gini_values


gini_values = compute_gini(combined_dataset)
combined_dataset['gini_coefficient'] = 0

for i, gini_value in enumerate(gini_values):
    meeting_number = meeting_numbers[i]
    combined_dataset.loc[
        (combined_dataset['project'] == 4) &
        (combined_dataset['meeting_number'] == meeting_number),
        'gini_coefficient'] = gini_value

C:\Users\sjy99\AppData\Local\Temp\ipykernel_28260\2128192228.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1999999750000031' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_dataset.loc[


## Compute Interaction Equality Index


In [28]:
# Define Interaction Equality Index function
def interaction_equality_index(x):
    x = np.array(x, dtype=np.float64)
    mean_x = np.mean(x)
    if mean_x == 0:
        return 0
    return 1 - (np.std(x) / mean_x)

# Compute Interaction Equality Index for each meeting


def compute_equality_index(df):
    equality_index_values = []
    meetings = df['meeting_number'].unique()
    for meeting_number in meetings:
        meeting_data = df[df['meeting_number'] == meeting_number]
        interaction_counts = [meeting_data[(meeting_data['speaker_number'] == speaker) & (
            meeting_data['speaker_number'] != meeting_data['next_speaker_id'])]['count'].sum() for speaker in meeting_data['speaker_number'].unique()]
        equality_index_values.append(
            interaction_equality_index(interaction_counts))
    return equality_index_values


equality_index = compute_equality_index(combined_dataset)
combined_dataset['interaction_equality_index'] = 0

for i, equality_index in enumerate(equality_index):
    meeting_number = meeting_numbers[i]
    combined_dataset.loc[
        (combined_dataset['project'] == 4) &
        (combined_dataset['meeting_number'] == meeting_number),
        'interaction_equality_index'] = equality_index

C:\Users\sjy99\AppData\Local\Temp\ipykernel_28260\3739568237.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  combined_dataset.loc[


## Save Updated Combined Dataset to CSV


In [29]:
# Reorder columns
import os


columns_order = [
    'id', 'project', 'meeting_number', 'speaker_number', 'speech_frequency', 'total_words', 'duration', 'normalized_speech_frequency', 'speaker_id', 'next_speaker_id', 'count', 'network_density', 'weighted_network_density',
    'gini_coefficient', 'interaction_equality_index', 'degree_centrality', 'indegree_centrality', 'outdegree_centrality', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality', 'pagerank'
]
combined_dataset = combined_dataset[columns_order]

# Save the final dataset with centralities and density to a CSV file
os.makedirs('data', exist_ok=True)
combined_dataset.to_csv('data/kakao_data.csv', index=False)